In [279]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cluster import KMeans

dataset = pd.read_csv("..\\Dataset\\food.csv")
#Seprating dataset into input and output values
#x = dataset[['bmi','age','vegNonveg','gender','cat','calorie_intake','food']]
#print(x.to_string())
x = dataset[['food']]
print(x)


                                                  food
0    Breakfast:Seedy Avocado Toast- Lunch: Veggie a...
1    Breakfast:Pear Banana Smoothie, Whole Wheat To...
2    Breakfast:Oatmeal ,Banana ,Peanut Butter, Choc...
3    Breakfast:Banana oatmeal smoothie - Lunch:Big ...
4    Breakfast:Banana oatmeal smoothie - Lunch:Big ...
..                                                 ...
194  Breakfast: Chicken and Cottage Cheese Sandwich...
195  Breakfast: Tropical protein smoothie- Lunch: C...
196  Breakfast: Chicken and Cottage Cheese Sandwich...
197  Breakfast: Chicken and Cottage Cheese Sandwich...
198  Breakfast: Spinach, Swiss, and Egg White Omele...

[199 rows x 1 columns]


In [254]:
print("TRAIN: \n", x.value_counts().to_string(), end="\n\n")


TRAIN: 
 food                                                                                                                                                                                                            
Breakfast:Banana oatmeal smoothie - Lunch:Big PB&J Sandwich,Carrots with Hummus - Dinner:5 Ingredient Coconut Curry                                                                                                 2
Breakfast: 1 Protein Energy Balls- Lunch: Corn Tuna Salad- Dinner: Easy Microwave Sausage, Kale, and White Beans                                                                                                    1
Breakfast: Strawberry Oatmeal- Lunch: ALT (Avocado, Lettuce, and Tomato) Sandwiches- Dinner: Grilled Cheese with Tomato, Peppers and Basil                                                                          1
Breakfast: Spiced Sweet Potato and Goat Cheese Egg Skillet - Lunch: 2 Strawberry Smoothies - Dinner: 2 Cheese Spinach Turkey Burgers        

In [255]:
from sklearn.preprocessing import LabelEncoder

# Creazione dell'oggetto LabelEncoder
label_encoder = LabelEncoder()

# Codifica delle variabili categoriche come valori numerici
X_encoded = x.copy()  # Copia dei dati originali
X_encoded['food'] = label_encoder.fit_transform(x['food'])


In [256]:
from sklearn.preprocessing import OneHotEncoder

# Creazione dell'oggetto OneHotEncoder
encoder = OneHotEncoder()
X_encoded = X_encoded.values
# Adattamento del modello agli dati
encoder.fit(X_encoded)

# Trasformazione dei dati di addestramento
X_encoded = encoder.transform(X_encoded).toarray()


In [257]:
from sklearn.preprocessing import StandardScaler

# Creazione di un oggetto StandardScaler
scaler = StandardScaler()

# Adattamento del modello agli dati
scaler.fit(X_encoded)

# Trasformazione dei dati di addestramento
X_scaled = scaler.transform(X_encoded)



In [269]:
kmeans = KMeans(n_clusters=70, random_state=0).fit(X_scaled)
print(kmeans.labels_)

[58 34 48 34 47 34 27 34 34 34 34 41 34 44 29 34 69 34 34 40 63 21 38 49
 34 34 34 34 34 64 34 34 34 34 55 34 34 20 34 34 34 34 51 34 37 34 34 34
 34 34 34 34 42 54 34 12 34 34 34 34 32 61 56 45 17 34 34  6 43 34 34 34
  8 34 34 34 39 34 14 34  2  5 34 34 34 34 34 34 34 34 62 34 34 34 34 34
 34 33 34 10 15 34 24 65 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 46
 34 34  1 67 34 34 22 34 34 34 34 34  4 18 34 36 34 34 34 66 19 34 34 34
 34 68 34 34  9 11 34 34 34 60 34  3 34 34 34 34 34 34 23 34 34 34 34 50
 34 57 34 34 34 28 34 25 34 13 34 30 34 34  0 26 16 52 34 34 34 34 59 34
 34 35 34 53 34  7 31]


In [270]:
x = dataset[['bmi','age','vegNonveg','gender','cat','calorie_intake']]
y = kmeans.labels_

In [271]:
#Splitting dataset into train and test
x_train,x_test, y_train, y_test= train_test_split(x,y, test_size=0.3,random_state=0)

In [272]:
model = RandomForestClassifier(criterion='entropy', warm_start = True)
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', warm_start=True)

In [90]:
from sklearn import svm
from sklearn.svm import LinearSVC


model = svm.LinearSVC(max_iter = 100000000)
model.fit(x_train,y_train)


LinearSVC(max_iter=100000000)

In [273]:
def evaluation_metrics(x, y, model):

    y_pred = model.predict(x)

    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, average="macro",labels=np.unique(y_pred))
    recall = recall_score(y, y_pred, average="macro",labels=np.unique(y_pred))
    f1 = f1_score(y, y_pred, average="macro",labels=np.unique(y_pred))

    print("Accuracy: {:.4f}".format(accuracy))
    print("Precision: {:.4f}".format(precision))
    print("Recall: {:.4f}".format(recall))
    print("F1 Score: {:.4f}".format(f1))

In [76]:
import warnings
warnings.filterwarnings('always')

In [163]:
print(X_encoded)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [274]:
evaluation_metrics(x_test, y_test, model)


Accuracy: 0.6500
Precision: 0.6500
Recall: 1.0000
F1 Score: 0.7879


In [275]:
x_test= [[23,44,0,0,1,1900]]
x = dataset[['bmi','age','vegNonveg','gender','cat','calorie_intake']]

y_pred = model.predict(x_test)

C:\Users\perna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [276]:
print(y_pred)

[34]


In [277]:
x = dataset[['food']]
list = x.values

In [278]:
print(list[y_pred])

[['Breakfast:TChocolate peanut protein shake - Lunch:Hummus on Whole Wheat- Dinner:Soba Noodles & Coconut Curry,Lemon Steamed Broccoli']]
